In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math

from sklearn.metrics import confusion_matrix

## 1. Data preprocessing

### 1.1 read data

In [2]:
defaultVal = [[0.] for idx in range(2*3 + 1)]
print(defaultVal)

[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]


In [3]:
# get the data of the subject and returns df with the relevant information
def prep_data(path, subject):
    data = pd.read_csv(path, sep=' ', header=None)
    data.columns = pd.RangeIndex(1, len(data.columns) + 1) 
    data.drop(3, axis='columns', inplace=True)
    data = data.dropna()
    data = data[[1,2,22,23,24,28,29,30]]
    cols = {1: 'time_step', 2: 'activity_id', 22: 'acc_x', 23: 'acc_y', 24: 'acc_z', 28: 'gyro_x', 29: 'gyro_y', 30: 'gyro_z'}
    data = data.rename(columns=cols)
    # calculating norm
    data['acc_norm'] = np.sqrt(data['acc_x'] ** 2 + data['acc_y'] ** 2 + data['acc_z'] ** 2)
    data['gyro_norm'] = np.sqrt(data['gyro_x'] ** 2 + data['gyro_y'] ** 2 + data['gyro_z'] ** 2)
    data['User'] = f'{subject}'
    print(f'{subject}')
    return data

In [ ]:
# data = pd.read_csv('/Users/momo/Documents/dataset/pamap2+physical+activity+monitoring/Protocol/subject101.dat', sep=' ', header=None)
# data.head()

In [20]:
# Perform preprocess for different subjects
# subjects = ['101', '102', '105', '106', '107', '108']
subjects = ['101','102']
path = '/Users/momo/Documents/dataset/pamap2+physical+activity+monitoring/Protocol'
data_list = []
for subject in subjects:
    data_list.append(prep_data(path + '/subject' + subject + '.dat', subject))

101
102


In [22]:
data_list[0].head(),data_list[1].head()

(   time_step  activity_id     acc_x    acc_y    acc_z    gyro_x    gyro_y  \
 0       8.38            0  0.238080  9.80003 -1.68896 -0.005065 -0.006781   
 1       8.39            0  0.319530  9.61282 -1.49328  0.013685  0.001486   
 2       8.40            0  0.235593  9.72421 -1.76621 -0.039923  0.034056   
 3       8.41            0  0.388697  9.53572 -1.72410  0.007513 -0.010498   
 4       8.42            0  0.315800  9.49908 -1.60914 -0.003822 -0.011217   
 
      gyro_z  acc_norm  gyro_norm User  
 0 -0.005663  9.947354   0.010184  101  
 1 -0.041522  9.733360   0.043744  101  
 2 -0.002113  9.886115   0.052518  101  
 3 -0.020684  9.698122   0.024382  101  
 4 -0.025975  9.639584   0.028550  101  ,
    time_step  activity_id    acc_x    acc_y    acc_z    gyro_x    gyro_y  \
 0       5.64            0  1.94739  9.59644 -3.12873  0.124025  0.112482   
 1       5.65            0  1.75120  9.63340 -3.32601  0.132679  0.060829   
 2       5.66            0  1.67059  9.70790 -3.4826

In [24]:
# Combine to one df
for d in range(len(data_list)):
  if d==0:
    df = data_list[0]
  else:
    df = pd.concat([df, data_list[d]], axis=0, ignore_index=True)

### 1.2 normalization

In [25]:
def normalize(df, cols): #list of columns
    df_t=(df[cols]-df[cols].mean())/df[cols].std() #均值为0，方差为1的分布
    df_norm = df.copy()
    df_norm[cols] = df_t

    return df_norm

In [26]:
data_norm = normalize(df, ['acc_norm', 'gyro_norm'])

### 1.3 augmentation

In [39]:
def add_noise(df, cols, sigma=0.1):
    noise = np.random.normal(0, sigma, df[cols].shape)
    new_signal = df[cols] + noise
    df_noise = df.copy()
    df_noise[cols] = new_signal

    return df_noise

def augment(df, cols, num_of_inst):
    '''
    df should be inserted normalized
    num_of_inst = num of instances to create from each instance
    '''
    aug_df = df.copy()
    for i in range(num_of_inst):
        np.random.seed(i)
        nois_data = add_noise(df, cols)
        nois_data['User'] = nois_data['User'] + f'{i}'
        aug_df= pd.concat([aug_df, nois_data], axis=0, ignore_index=True)

    return aug_df

In [40]:
data_aug = augment(data_norm, ['acc_norm', 'gyro_norm'], 1)

In [47]:
data_aug.head()

,time_step,activity_id,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,acc_norm,gyro_norm,User
0,8.38,0,0.238080,9.80003,-1.68896,-0.005065,-0.006781,-0.005663,-0.066101,-0.818660,101
1,8.39,0,0.319530,9.61282,-1.49328,0.013685,0.001486,-0.041522,-0.128133,-0.764882,101
2,8.40,0,0.235593,9.72421,-1.76621,-0.039923,0.034056,-0.002113,-0.083853,-0.750823,101
3,8.41,0,0.388697,9.53572,-1.72410,0.007513,-0.010498,-0.020684,-0.138347,-0.795908,101
4,8.42,0,0.315800,9.49908,-1.60914,-0.003822,-0.011217,-0.025975,-0.155316,-0.789230,101


### 1.4 FFT

In [48]:
aaa = data_aug
np.unique(aaa['activity_id'])
np.unique(aaa['User'])

array(['101', '1010', '102', '1020'], dtype=object)

In [49]:
initial_data = data_aug
unique_actions = np.unique(initial_data['activity_id'])
unique_users = np.unique(initial_data['User'])
dff = pd.DataFrame(columns=['activity','timstep', 'user','acc_spec','gyro_spec'])
print(dff.shape)
dff.head()

((0, 5),
 Empty DataFrame
 Columns: [activity, timstep, user, acc_spec, gyro_spec]
 Index: [])

In [52]:
print(unique_actions)

[ 0  1  2  3  4  5  6  7 12 13 16 17 24]


In [74]:
# path = '/content/drive/MyDrive/Advanced ML final project/sepc_6_users'
for action in [1]:
    print('action: ', action)
    for user in ['101']:
        print('user: ', user)
        act_user_temp = initial_data[(initial_data['activity_id']==action) & (initial_data['User']==user)]
        NFFT = 25
        noverlap = int(0.25 * NFFT)
        print(act_user_temp.shape, noverlap)
        fig = plt.figure(frameon=False)
        sr = 100
        timestep = 0
        for row in range(0, act_user_temp.shape[0], 150): # overlap of the data is 50%. not the hyperparameter of FFT.
            timestep += 1
            spec_acc, freqenciesFound_x, time, imageAxis_x = plt.specgram(act_user_temp['acc_norm'][row:row + 200], Fs=100, NFFT=NFFT, noverlap=noverlap, window=np.hamming(NFFT),cmap='viridis')
            spec_gyro, freqenciesFound_x, time, imageAxis_x = plt.specgram(act_user_temp['gyro_norm'][row:row + 200], Fs=100, NFFT=NFFT, noverlap=noverlap, window=np.hamming(NFFT),cmap='viridis')
            instance = pd.DataFrame({'activity':action, 'timstep':timestep, 'user':user, 'acc_spec':[spec_acc], 'gyro_spec':[spec_gyro]})
            dff = dff.append(instance, ignore_index=True) # label, timestep, user, f*t (spectrogram), f*t (spectrogram)
        plt.close("all")

action:  1
user:  101
(27179, 11) 6


In [ ]:
# dff.to_pickle("path/labeled_data_6_users_2_sec.pkl")
# df = pd.read_pickle("path/labeled_data_6_users_2_sec.pkl")

### 1.5 filtering targeted activities and making the labels sequencials

In [ ]:
# df = df[df['activity'].isin([1,2,3,4,5,6,7,12,13,16,17])]
# df['activity'].loc[(df['activity']==12)] = 8
# df['activity'].loc[(df['activity']==13)] = 9
# df['activity'].loc[(df['activity']==16)] = 10
# df['activity'].loc[(df['activity']==17)] = 0
# df[df['activity'] == 10].shape

## 2. Model

In [ ]:
SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 8
CONV_MERGE_LEN2 = 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 120
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_KEEP_PROB = 0.8

BATCH_SIZE = 64
TOTAL_ITER_NUM = 30000

select = 'a'

metaDict = {'a':[119080, 1193], 'b':[116870, 1413], 'c':[116020, 1477]}
TRAIN_SIZE = metaDict[select][0]
EVAL_DATA_SIZE = metaDict[select][1]
EVAL_ITER_NUM = int(math.ceil(EVAL_DATA_SIZE / BATCH_SIZE))

In [ ]:
class BatchNormLayer(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(BatchNormLayer, self).__init__()
        self.bn = nn.BatchNorm1d(num_features, eps=eps, momentum=momentum)

    def forward(self, inputs, phase_train):
        if phase_train:
            return self.bn(inputs)
        else:
            return self.bn(inputs)

# Usage:
# phase_train = True for training, and False for evaluation
# Example usage: batch_norm_layer = BatchNormLayer(num_features=32, eps=1e-5, momentum=0.1)
#                output = batch_norm_layer(input_tensor, phase_train)

In [ ]:
def sensor_local_feature_layer(inputs, name, train):
    # CONV_NUM is 64
    conv1 = nn.Conv2d(inputs.size(1), CONV_NUM, kernel_size=(1, 2*3*CONV_LEN), stride=(1, 2*3), padding=0)
    conv1 = nn.BatchNorm2d(CONV_NUM)
    conv1_shape = conv1(inputs).size()
    print("conv1 ", conv1_shape)
    conv1 = nn.ReLU(inplace=True)
    conv1 = nn.Dropout2d(p=1-CONV_KEEP_PROB)(conv1(inputs))

    # CONV_LEN_INTE is 3
    conv2 = nn.Conv2d(CONV_NUM, CONV_NUM, kernel_size=(1, CONV_LEN_INTE), stride=(1, 1), padding=0)
    conv2 = nn.BatchNorm2d(CONV_NUM)
    conv2_shape = conv2(conv1).size()
    print("conv2 ", conv2_shape)
    conv2 = nn.ReLU(inplace=True)
    conv2 = nn.Dropout2d(p=1-CONV_KEEP_PROB)(conv2(conv1))

    # CONV_LEN_LAST is 3
    conv3 = nn.Conv2d(CONV_NUM, CONV_NUM, kernel_size=(1, CONV_LEN_LAST), stride=(1, 1), padding=0)
    conv3 = nn.BatchNorm2d(CONV_NUM)
    conv3_shape = conv3(conv2).size()
    print("conv3 ", conv3_shape)

    conv_out = conv3.view(conv3_shape[0], conv3_shape[1], 1, conv3_shape[2], conv3_shape[3])
    return conv_out


In [ ]:
class CNN_acc(nn.Module):
    def __init__(self):
        super(CNN_acc, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (1,3), padding='same')
        self.conv2 = nn.Conv2d(32, 32, (1,3), padding='same')
        self.conv3 = nn.Conv2d(32, 64, (1,3), padding='same')
        self.conv4 = nn.Conv2d(64, 64, (1,3), padding='same')
        self.fc = nn.Linear(2048, 128)
        torch.nn.init.xavier_normal(self.fc.weight)
        torch.nn.init.xavier_normal(self.conv1.weight)
        torch.nn.init.xavier_normal(self.conv2.weight)
        torch.nn.init.xavier_normal(self.conv3.weight)
        torch.nn.init.xavier_normal(self.conv4.weight)


    def forward(self, x):
        # print('x.shape 1: ', x.shape)
        x = F.relu(self.conv1(x))
        # print('x.shape 2: ', x.shape)
        x = F.relu(F.max_pool2d(self.conv2(x), (1,2)))
        # print('x.shape 3: ', x.shape)
        x = F.relu(self.conv3(x))
        # print('x.shape 4: ', x.shape)
        x = F.relu(F.max_pool2d(self.conv4(x), (1,2)))
        # print('x.shape 5: ', x.shape)
        x = torch.flatten(x, start_dim=1)
        # print('x.shape 6: ', x.shape)
        x = self.fc(x)
        # print('x.shape 7: ', x.shape)

        return x

In [3]:
inputs= np.ones(120).reshape([2, 3, 4, 5])
inputs = torch.from_numpy(inputs)

In [ ]:
def attention_fusion_1(inputs): # Attention-fusion Subnet
    """inputs, shape: [batch, time_step, input_mode, feature_dim]"""
    d = inputs.shape[-1]
    w = torch.randn(d, requires_grad=True, dtype=torch.double) * 0.1 # weights
    b = torch.randn(1, requires_grad=True, dtype=torch.double) 

    activation = torch.tanh(torch.matmul(inputs, w) + b)  # b * t * i
    alphas = F.softmax(activation, dim=-1)  # b * t * i
    print("alphas ", str(alphas.shape))

    output = torch.sum(inputs * alphas.unsqueeze(-1), dim=2)
    print(output.shape) #shape: [batch, time_step, 1, feature_dim]
    return output

In [ ]:
def attention(inputs, attention_size, time_major=False, return_alphas=False):
    if isinstance(inputs, tuple):
        # In case of Bi-RNN, concatenate the forward and the backward RNN outputs.
        inputs = torch.cat(inputs, 2)

    if time_major:
        # (T,B,D) => (B,T,D)
        inputs = inputs.transpose(0, 1)

    hidden_size = inputs.size(2)  # D value - hidden size of the RNN layer

    # Trainable parameters
    w_omega = nn.Parameter(torch.randn(hidden_size, attention_size) * 0.1)
    b_omega = nn.Parameter(torch.randn(attention_size) * 0.1)
    u_omega = nn.Parameter(torch.randn(attention_size) * 0.1)

    with torch.no_grad():
        w_omega.requiresGrad = True
        b_omega.requiresGrad = True
        u_omega.requiresGrad = True

    with torch.name_scope('v'):
        # Applying fully connected layer with non-linear activation to each of the B*T timestamps;
        # the shape of `v` is (B,T,D)*(D,A)=(B,T,A), where A=attention_size
        v = torch.tanh(torch.matmul(inputs, w_omega) + b_omega)

    # For each of the timestamps, its vector of size A from `v` is reduced with the `u` vector
    vu = torch.matmul(v, u_omega).squeeze(-1)  # (B,T) shape
    alphas = F.softmax(vu, dim=1)   # (B,T) shape

    # Output of (Bi-)RNN is reduced with the attention vector; the result has (B,D) shape
    output = torch.sum(inputs * alphas.unsqueeze(-1), dim=1)

    if not return_alphas:
        return output
    else:
        return output, alphas


In [ ]:
def self_attention(inputs, name):
    """
    :param inputs: input tensor (B, T, 3, dim)
    :param name: scope name
    :return: final_output (B, T, dim)
    """
    t = inputs.size(1)
    share_param = True
    hidden_size = inputs.size(-1)  # D value - hidden size of the RNN layer

    if share_param:
        scope_name = 'self_attn'
    else:
        scope_name = 'self_attn' + name

    inputs = inputs.transpose(1, 0)  # (T, B, 3, dim)

    outputs = []
    for x in range(t):
        t_x = inputs[x]  # (B, 3, dim)

        den = True
        if den:
            x_proj = nn.Linear(hidden_size, hidden_size)(t_x)
            x_proj = torch.tanh(x_proj)
        else:
            x_proj = t_x

        u_w = nn.Parameter(torch.randn(hidden_size, 1) * 0.01, requires_grad=True)
        x = torch.matmul(x_proj, u_w)  # (B, 3, 1)
        alphas = F.softmax(x, dim=1)  # (B, 3, 1)

        output = torch.matmul(t_x.transpose(1, 2), alphas)  # (B, dim, 1)
        output = output.squeeze(-1)  # (B, dim)
        outputs.append(output)

    final_output = torch.stack(outputs, dim=1)  # (B, T, dim)

    return final_output

In [ ]:
class DeepSense(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, train=True, name='deepSense'):
        super(DeepSense, self).__init__()
        self.train = train
        self.name = name
        self.used = None
        self.avgNum = None

        # Define other layers here if needed

        self.gru_cell1 = nn.GRUCell(input_size, hidden_size)
        if train:
            self.gru_cell1 = nn.Dropout(p=0.5)

        self.gru_cell2 = nn.GRUCell(hidden_size, hidden_size)
        if train:
            self.gru_cell2 = nn.Dropout(p=0.5)

        self.cell = nn.RNNCellBase([self.gru_cell1, self.gru_cell2])
        self.init_state = torch.zeros(2, BATCH_SIZE, hidden_size, dtype=torch.float32)

        self.AZ = 80
        self.attention_out = None

    def forward(self, inputs):
        used = torch.sign(torch.max(torch.abs(inputs), dim=2)[0])  # (BATCH_SIZE, WIDE)
        self.used = used
        length = torch.sum(used, dim=1)  # (BATCH_SIZE)
        length = length.type(torch.int64)

        mask = torch.sign(torch.max(torch.abs(inputs), dim=2, keepdim=True)[0])
        mask = mask.repeat(1, 1, INTER_DIM)  # (BATCH_SIZE, WIDE, INTER_DIM)
        avgNum = torch.sum(mask, dim=1)  # (BATCH_SIZE, INTER_DIM)
        self.avgNum = avgNum

        sensor_inputs = inputs.unsqueeze(3)  # (BATCH_SIZE, WIDE, FEATURE_DIM, CHANNEL=1)
        acc_inputs, gyro_inputs = torch.split(sensor_inputs, split_size_or_sections=input_size // 2, dim=2)

        acc_conv_out = self.sensor_local_feature_layer(acc_inputs, "acc", train=self.train)
        gyro_conv_out = self.sensor_local_feature_layer(gyro_inputs, "gyro", train=self.train)

        sensor_conv_in = torch.cat([acc_conv_out, gyro_conv_out], dim=2)
        sensor_conv3 = sensor_conv_in

        attention_input = sensor_conv3.view(sensor_conv3.size(0), sensor_conv3.size(1), sensor_conv3.size(2),
                                            sensor_conv3.size(3) * sensor_conv3.size(4))
        sensor_conv_out = self.attention_fusion_1(attention_input)

        self.init_state = torch.zeros(2, inputs.size(0), self.cell.hidden_size, dtype=torch.float32)
        cell_output = []
        h_t = self.init_state

        for t in range(inputs.size(1)):
            h_t = self.cell(inputs[:, t], h_t)
            cell_output.append(h_t)

        cell_output = torch.stack(cell_output, dim=1)  # (BATCH_SIZE, WIDE, INTER_DIM)

        attention_out = self.attention(cell_output, attention_size=self.AZ)
        self.attention_out = attention_out

        avg_cell_out = attention_out

        # Define other operations here if needed

        # Assuming fully connected layer for logits
        logits = self.fully_connected_layer(avg_cell_out)

        return logits


# Usage:
# Create an instance of DeepSense with appropriate input_size, hidden_size, output_size, and train (True for training, False for evaluation)
# deep_sense_model = DeepSense(input_size=..., hidden_size=..., output_size=..., train=...)
# logits = deep_sense_model(input_tensor)


In [ ]:
# Create an instance of the DeepSense model with training mode
deep_sense_model = DeepSense(input_size=..., hidden_size=..., output_size=..., train=True)

# Forward pass to get model
model = deep_sense_model(batch_feature)

# Calculate the predictions
_, predict = torch.max(model, dim=1)

# Calculate the cross-entropy loss
criterion = nn.CrossEntropyLoss()
batchLoss = criterion(model, batch_label)
loss = torch.mean(batchLoss)

# Create an instance of the DeepSense model with evaluation mode (reuse=True)
deep_sense_model_eval = DeepSense(input_size=..., hidden_size=..., output_size=..., train=False, reuse=True)

# Forward pass to get model for evaluation
model_eval = deep_sense_model_eval(batch_eval_feature)

# Calculate the predictions for evaluation
_, predict_eval = torch.max(model_eval, dim=1)

# Calculate the cross-entropy loss for evaluation
loss_eval = torch.mean(criterion(model_eval, batch_eval_label))

# Get the trainable parameters of the model
t_params = deep_sense_model.parameters()

# Calculate the L2 regularization term
regularizers = torch.tensor(0., dtype=torch.float32)
for param in t_params:
    regularizers += torch.sum(param * param)  # L2 loss
loss += 5e-4 * regularizers


In [ ]:
# Create an instance of the DeepSense model with training mode
deep_sense_model = DeepSense(input_size=..., hidden_size=..., output_size=..., train=True)

# Initialize model parameters
deep_sense_model.apply(weights_init)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deep_sense_model.to(device)

# Assuming you have defined the data loaders for training and evaluation
train_loader = ...
eval_loader = ...

best = -0.1
for iteration in range(TOTAL_ITER_NUM):
    deep_sense_model.train()  # Set the model to training mode

    for batch_feature, batch_label in train_loader:
        batch_feature = batch_feature.to(device)
        batch_label = batch_label.to(device)

        # Forward pass and calculate loss
        logits = deep_sense_model(batch_feature)
        loss = criterion(logits, batch_label)

        # Backpropagation and optimization
        discOptimizer.zero_grad()
        loss.backward()
        discOptimizer.step()

        # Assuming you have implemented a function to calculate accuracy
        accuracy = calculate_accuracy(logits, batch_label)

        # Plot training statistics (Note: Implement a similar plot function)
        plot.plot('train cross entropy', loss.item())
        plot.plot('train accuracy', accuracy)

    if iteration % 50 == 49:
        deep_sense_model.eval()  # Set the model to evaluation mode
        dev_accuracy = []
        dev_cross_entropy = []
        with torch.no_grad():
            for batch_eval_feature, batch_eval_label in eval_loader:
                batch_eval_feature = batch_eval_feature.to(device)
                batch_eval_label = batch_eval_label.to(device)

                # Forward pass for evaluation
                logits_eval = deep_sense_model(batch_eval_feature)
                eval_loss = criterion(logits_eval, batch_eval_label)

                # Calculate accuracy for evaluation
                eval_accuracy = calculate_accuracy(logits_eval, batch_eval_label)

                dev_accuracy.append(eval_accuracy)
                dev_cross_entropy.append(eval_loss.item())

        mean_dev_accuracy = torch.mean(torch.tensor(dev_accuracy))
        best = max(best, mean_dev_accuracy)
        plot.plot('dev accuracy', mean_dev_accuracy.item())
        plot.plot('dev cross entropy', torch.mean(torch.tensor(dev_cross_entropy)).item())

    if (iteration < 5) or (iteration % 50 == 49):
        plot.flush()

    plot.tick()

print("best score " + str(best))


## 3. Experiment

### 3.1 parameters

### 3.2 training

### 3.3 evaluation